INITIAL

In [0]:
file_path = "dbfs:/FileStore/tables/Files/actors.csv"
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import when, col
broken_file_path = "dbfs:/FileStore/tables/Files/corrupted_actors.csv"
spark = SparkSession.builder.appName("CSV Reader").getOrCreate()

ZADANIE2

In [0]:
schema = StructType([
    StructField("imdb_title_id", StringType(), True),
    StructField("ordering", IntegerType(), True),
    StructField("imdb_name_id", StringType(), True),
    StructField("category", StringType(), True),
    StructField("job", StringType(), True),
    StructField("characters", StringType(), True)
])

df = spark.read.csv(file_path, header=True, schema=schema, inferSchema=False)
df.show()

+-------------+--------+------------+---------------+----------+--------------------+
|imdb_title_id|ordering|imdb_name_id|       category|       job|          characters|
+-------------+--------+------------+---------------+----------+--------------------+
|    tt0000009|       1|   nm0063086|        actress|      null|[Miss Geraldine H...|
|    tt0000009|       2|   nm0183823|          actor|      null|      [Mr. Hamilton]|
|    tt0000009|       3|   nm1309758|          actor|      null|[Chauncey Depew -...|
|    tt0000009|       4|   nm0085156|       director|      null|                null|
|    tt0000574|       1|   nm0846887|        actress|      null|        [Kate Kelly]|
|    tt0000574|       2|   nm0846894|          actor|      null|     [School Master]|
|    tt0000574|       3|   nm3002376|          actor|      null|        [Steve Hart]|
|    tt0000574|       4|   nm0170118|        actress|      null|                null|
|    tt0000574|       5|   nm0846879|       director| 

ZADANIE3

CREATING BROKEN FILE CSV

In [0]:
df2 = df.withColumn(
    "ordering", 
    when((col("ordering") == 5) | (col("ordering") == 2), "XD").otherwise(col("ordering"))
)
df2.show()
df2.write.csv(broken_file_path , header=True, mode="overwrite")

+-------------+--------+------------+---------------+----------+--------------------+
|imdb_title_id|ordering|imdb_name_id|       category|       job|          characters|
+-------------+--------+------------+---------------+----------+--------------------+
|    tt0000009|       1|   nm0063086|        actress|      null|[Miss Geraldine H...|
|    tt0000009|      XD|   nm0183823|          actor|      null|      [Mr. Hamilton]|
|    tt0000009|       3|   nm1309758|          actor|      null|[Chauncey Depew -...|
|    tt0000009|       4|   nm0085156|       director|      null|                null|
|    tt0000574|       1|   nm0846887|        actress|      null|        [Kate Kelly]|
|    tt0000574|      XD|   nm0846894|          actor|      null|     [School Master]|
|    tt0000574|       3|   nm3002376|          actor|      null|        [Steve Hart]|
|    tt0000574|       4|   nm0170118|        actress|      null|                null|
|    tt0000574|      XD|   nm0846879|       director| 

PERMISSIVE

In [0]:
df_permissive = spark.read.csv(broken_file_path, header=True, schema=schema, mode="PERMISSIVE")
df_permissive.show()

+-------------+--------+------------+---------------+----------+--------------------+
|imdb_title_id|ordering|imdb_name_id|       category|       job|          characters|
+-------------+--------+------------+---------------+----------+--------------------+
|    tt0000009|       1|   nm0063086|        actress|      null|[Miss Geraldine H...|
|    tt0000009|    null|   nm0183823|          actor|      null|      [Mr. Hamilton]|
|    tt0000009|       3|   nm1309758|          actor|      null|[Chauncey Depew -...|
|    tt0000009|       4|   nm0085156|       director|      null|                null|
|    tt0000574|       1|   nm0846887|        actress|      null|        [Kate Kelly]|
|    tt0000574|    null|   nm0846894|          actor|      null|     [School Master]|
|    tt0000574|       3|   nm3002376|          actor|      null|        [Steve Hart]|
|    tt0000574|       4|   nm0170118|        actress|      null|                null|
|    tt0000574|    null|   nm0846879|       director| 

DROPMALFORMED

In [0]:
df_dropmalformed = spark.read.csv(broken_file_path, header=True, schema=schema, mode="DROPMALFORMED")
df_dropmalformed.show()

+-------------+--------+------------+---------------+----------+--------------------+
|imdb_title_id|ordering|imdb_name_id|       category|       job|          characters|
+-------------+--------+------------+---------------+----------+--------------------+
|    tt0000009|       1|   nm0063086|        actress|      null|[Miss Geraldine H...|
|    tt0000009|       3|   nm1309758|          actor|      null|[Chauncey Depew -...|
|    tt0000009|       4|   nm0085156|       director|      null|                null|
|    tt0000574|       1|   nm0846887|        actress|      null|        [Kate Kelly]|
|    tt0000574|       3|   nm3002376|          actor|      null|        [Steve Hart]|
|    tt0000574|       4|   nm0170118|        actress|      null|                null|
|    tt0000574|       6|   nm0317210|       producer|  producer|                null|
|    tt0000574|       7|   nm0425854|       producer|  producer|                null|
|    tt0000574|       8|   nm0846911|       producer| 

FAILFAST

In [0]:
try:
    df_failfast = spark.read.csv(broken_file_path, header=True, schema=schema, mode="FAILFAST")
    df_failfast.show()
except Exception as e:
    print("WyPERDOLIO SIĘ COŚ")

WyPERDOLIO SIĘ COŚ


ZADANIE4

WRITE

In [0]:
df.write.parquet("dbfs:/FileStore/Tables/Files/parquet2.parquet", mode="overwrite")
df.write.json("dbfs:/FileStore/Tables/Files/json2.json", mode="overwrite")

READ

In [0]:
df_parquet = spark.read.parquet("dbfs:/FileStore/Tables/Files/parquet2.parquet")
df_parquet.show()

df_json = spark.read.json("dbfs:/FileStore/Tables/Files/json2.json")
df_json.show()

+-------------+--------+------------+-------------------+--------+------------+
|imdb_title_id|ordering|imdb_name_id|           category|     job|  characters|
+-------------+--------+------------+-------------------+--------+------------+
|    tt3249124|       4|   nm6005417|              actor|    null|       [Tom]|
|    tt3249124|       5|   nm1871431|           director|    null|        null|
|    tt3249124|       6|   nm1862032|           producer|producer|        null|
|    tt3249124|       7|   nm4261282|           composer|    null|        null|
|    tt3249124|       8|   nm1677303|production_designer|    null|        null|
|    tt3249158|       1|   nm2946712|              actor|    null| [Sebastian]|
|    tt3249158|       2|   nm2578315|            actress|    null|   [Miranda]|
|    tt3249158|       3|   nm7033017|              actor|    null|   [Jacinto]|
|    tt3249158|       4|   nm7033021|              actor|    null|      [Rosa]|
|    tt3249158|       5|   nm1848095|   